<a href="https://colab.research.google.com/github/golf41101/Suchet/blob/master/Recommendation_item_item_cosineSimilality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import os
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 20)

## Load Data

In [4]:
from google.colab import drive
#mount you GoogleDrive to "drive" folder
drive.mount('/drive', force_remount=False)

Mounted at /drive


In [5]:
#set path
path ="/drive/MyDrive/Recommendation/"
os.chdir(path)

In [6]:
!ls

movie.csv  rating.csv  ratings_small.csv


In [7]:
rating = pd.read_csv(path + "rating.csv")

In [8]:
movie = pd.read_csv(path + "movie.csv")

In [9]:
movie.drop(columns='genres',inplace=True)

In [10]:
df = rating.merge(movie,on='movieId')

In [11]:
df.drop(columns='timestamp',inplace=True)

## Data Understanding

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
 3   title    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 762.9+ MB


In [13]:
df.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [14]:
df.head()

,userId,movieId,rating,title
0,1,2,3.5,Jumanji (1995)
1,5,2,3.0,Jumanji (1995)
2,13,2,3.0,Jumanji (1995)
3,29,2,3.0,Jumanji (1995)
4,34,2,3.0,Jumanji (1995)


## EDA

In [15]:
print(df['userId'].unique())

print(df['userId'].nunique())

[     1      5     13 ...  40432    861 121610]
138493


userID is [1,138493] : n,n+1,...

In [16]:
print(df['movieId'].unique())

print(df['movieId'].nunique())

[     2     29     32 ... 121021 110167 110510]
26744


movieID is not sereis number

### Give userId and movieId start to 0 and rearrange movieId [0,N-1]

In [17]:
# rating['userId'] = rating['userId'] - 1

In [18]:
# print(rating['userId'].unique())

# print(rating['userId'].nunique())

In [19]:
# rating['movieId'].values

In [20]:
# # create a mapping for movie ids
# unique_movie_ids = set(rating['movieId'].values)
# movie2idx = {}
# count = 0
# for movie_id in unique_movie_ids:
#   movie2idx[movie_id] = count
#   count += 1

In [21]:
# rating['movie_idx'] = rating.apply(lambda row: movie2idx[row.movieId], axis=1)

In [22]:
# from surprise import accuracy, SVD,Reader,Dataset
# from surprise.model_selection import train_test_split

# reader = Reader()
# df.head()

In [23]:
# it take so much rams,then i select 100000 data to this tasks

In [24]:
df.shape

(20000263, 4)

In [25]:
rating_small = df.iloc[0:1000000]

In [26]:
rating_small

,userId,movieId,rating,title
0,1,2,3.5,Jumanji (1995)
1,5,2,3.0,Jumanji (1995)
2,13,2,3.0,Jumanji (1995)
3,29,2,3.0,Jumanji (1995)
4,34,2,3.0,Jumanji (1995)
...,...,...,...,...
999995,90990,1198,3.5,Raiders of the Lost Ark (Indiana Jones and the...
999996,90991,1198,5.0,Raiders of the Lost Ark (Indiana Jones and the...
999997,90995,1198,5.0,Raiders of the Lost Ark (Indiana Jones and the...
999998,90997,1198,3.0,Raiders of the Lost Ark (Indiana Jones and the...


## Item-Based Filtering 

In [27]:
rating_mt = rating_small.pivot(index='userId',columns='title',values='rating').fillna(0)
rating_mt.head()

title,2001: A Space Odyssey (1968),Blade Runner (1982),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Clerks (1994),Die Hard (1988),Dragonheart (1996),E.T. the Extra-Terrestrial (1982),Escape to Witch Mountain (1975),"Fish Called Wanda, A (1988)",Interview with the Vampire: The Vampire Chronicles (1994),...,Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)",What's Eating Gilbert Grape (1993),"Wizard of Oz, The (1939)"
userId,,,,,,,,,,,,,,,,,,,,,
1,3.5,4.0,3.5,4.0,4.0,3.0,4.0,3.5,4.0,4.0,...,3.5,4.0,3.5,4.0,4.5,3.5,3.5,3.5,3.5,3.5
2,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
3,5.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,4.0,0.0,...,0.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,3.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,...,0.0,5.0,3.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0


In [28]:
rating_mt.shape

(125917, 32)

In [29]:
# movie_watched = rating_mt["Seven (a.k.a. Se7en) (1995)"]
# similarity_with_other_movies = rating_mt.corrwith(movie_watched)

# similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
# similarity_with_other_movies.head()

In [30]:
# rating_mt.reset_index(drop=True,inplace=True)

In [31]:
rating_mt = rating_mt.rename_axis(None, axis=1)

In [32]:
rating_mt.columns

Index(['2001: A Space Odyssey (1968)', 'Blade Runner (1982)',
       'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
       'Clerks (1994)', 'Die Hard (1988)', 'Dragonheart (1996)',
       'E.T. the Extra-Terrestrial (1982)', 'Escape to Witch Mountain (1975)',
       'Fish Called Wanda, A (1988)',
       'Interview with the Vampire: The Vampire Chronicles (1994)',
       'Jumanji (1995)',
       'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
       'Mask, The (1994)', 'Monty Python and the Holy Grail (1975)',
       'Monty Python's Life of Brian (1979)',
       'One Flew Over the Cuckoo's Nest (1975)', 'Platoon (1986)',
       'Pulp Fiction (1994)',
       'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
       'Reservoir Dogs (1992)', 'Rob Roy (1995)',
       'Rumble in the Bronx (Hont faan kui) (1995)',
       'Seven (a.k.a. Se7en) (1995)', 'Shawshank Redemption, The (1994)',
       'Silence of the Lambs, The (199

In [33]:
rating_mt.head()

,2001: A Space Odyssey (1968),Blade Runner (1982),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Clerks (1994),Die Hard (1988),Dragonheart (1996),E.T. the Extra-Terrestrial (1982),Escape to Witch Mountain (1975),"Fish Called Wanda, A (1988)",Interview with the Vampire: The Vampire Chronicles (1994),...,Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)",What's Eating Gilbert Grape (1993),"Wizard of Oz, The (1939)"
userId,,,,,,,,,,,,,,,,,,,,,
1,3.5,4.0,3.5,4.0,4.0,3.0,4.0,3.5,4.0,4.0,...,3.5,4.0,3.5,4.0,4.5,3.5,3.5,3.5,3.5,3.5
2,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
3,5.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,4.0,0.0,...,0.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,3.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,...,0.0,5.0,3.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0


In [34]:
# Create a placeholder dataframe listing item vs. item
data_mv = pd.DataFrame(index=rating_mt.columns,columns=rating_mt.columns)

In [35]:
# data_mv.reset_index(inplace=True)

In [36]:
# data_mv = data_mv.rename_axis(None, axis=1)

In [37]:
data_mv.head()

,2001: A Space Odyssey (1968),Blade Runner (1982),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Clerks (1994),Die Hard (1988),Dragonheart (1996),E.T. the Extra-Terrestrial (1982),Escape to Witch Mountain (1975),"Fish Called Wanda, A (1988)",Interview with the Vampire: The Vampire Chronicles (1994),...,Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)",What's Eating Gilbert Grape (1993),"Wizard of Oz, The (1939)"
2001: A Space Odyssey (1968),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Blade Runner (1982),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"City of Lost Children, The (Cité des enfants perdus, La) (1995)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Clerks (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Die Hard (1988),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
data_mv.shape

(32, 32)

In [39]:
for i in range(0,len(data_mv.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_mv.columns)) :
      # Fill in placeholder with cosine similarities
      data_mv.iloc[i,j] = 1-cosine(rating_mt.iloc[:,i],rating_mt.iloc[:,j])
      
data_mv

,2001: A Space Odyssey (1968),Blade Runner (1982),"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Clerks (1994),Die Hard (1988),Dragonheart (1996),E.T. the Extra-Terrestrial (1982),Escape to Witch Mountain (1975),"Fish Called Wanda, A (1988)",Interview with the Vampire: The Vampire Chronicles (1994),...,Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)",What's Eating Gilbert Grape (1993),"Wizard of Oz, The (1939)"
2001: A Space Odyssey (1968),1,0.597193,0.296813,0.330009,0.438005,0.205815,0.501191,0.159119,0.431695,0.256546,...,0.355497,0.358497,0.409793,0.498691,0.526303,0.427525,0.441431,0.388336,0.252079,0.449676
Blade Runner (1982),0.597193,1,0.338415,0.370912,0.509353,0.237727,0.506432,0.156765,0.464483,0.298893,...,0.423229,0.415161,0.465283,0.564041,0.599028,0.498821,0.507109,0.469853,0.267815,0.413258
"City of Lost Children, The (Cité des enfants perdus, La) (1995)",0.296813,0.338415,1,0.243859,0.213668,0.140326,0.225992,0.094917,0.237104,0.182834,...,0.233655,0.201452,0.228285,0.26413,0.260469,0.236707,0.328645,0.254247,0.186448,0.202576
Clerks (1994),0.330009,0.370912,0.243859,1,0.3778,0.187862,0.35719,0.123296,0.362225,0.341074,...,0.429724,0.405242,0.41655,0.387165,0.398972,0.399569,0.414139,0.452969,0.343883,0.331717
Die Hard (1988),0.438005,0.509353,0.213668,0.3778,1,0.277985,0.528329,0.176853,0.469713,0.326648,...,0.478719,0.45227,0.490585,0.555534,0.59684,0.577328,0.45008,0.472021,0.264046,0.413165
Dragonheart (1996),0.205815,0.237727,0.140326,0.187862,0.277985,1,0.270864,0.171885,0.220915,0.240566,...,0.222325,0.212054,0.227961,0.353219,0.278016,0.289065,0.322743,0.200125,0.163254,0.222714
E.T. the Extra-Terrestrial (1982),0.501191,0.506432,0.225992,0.35719,0.528329,0.270864,1,0.226573,0.470957,0.324498,...,0.409184,0.432149,0.47062,0.560578,0.600938,0.488876,0.427068,0.413314,0.308624,0.537626
Escape to Witch Mountain (1975),0.159119,0.156765,0.094917,0.123296,0.176853,0.171885,0.226573,1,0.181945,0.124316,...,0.109808,0.114617,0.134313,0.184642,0.186508,0.150518,0.122547,0.116964,0.115598,0.236439
"Fish Called Wanda, A (1988)",0.431695,0.464483,0.237104,0.362225,0.469713,0.220915,0.470957,0.181945,1,0.269122,...,0.339404,0.349234,0.396993,0.426172,0.449794,0.377673,0.367361,0.384135,0.310771,0.423416
Interview with the Vampire: The Vampire Chronicles (1994),0.256546,0.298893,0.182834,0.341074,0.326648,0.240566,0.324498,0.124316,0.269122,1,...,0.51845,0.438285,0.492229,0.312678,0.309973,0.474331,0.444636,0.402381,0.368687,0.261358


In [40]:
tp_shape_data = data_mv.shape[0]

In [41]:
data_mv.iloc[1,2]

0.3384152577446917

In [42]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_mv.columns,columns=range(1,12))

In [43]:
data_neighbours

,1,2,3,4,5,6,7,8,9,10,11
2001: A Space Odyssey (1968),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Blade Runner (1982),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"City of Lost Children, The (Cité des enfants perdus, La) (1995)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Clerks (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Die Hard (1988),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dragonheart (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E.T. the Extra-Terrestrial (1982),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Escape to Witch Mountain (1975),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Fish Called Wanda, A (1988)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interview with the Vampire: The Vampire Chronicles (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_mv.columns)):
  data_neighbours.iloc[i,:tp_shape_data] = data_mv.iloc[0:,i].sort_values(ascending=False)[:11].index
 
# --- End Item Based Recommendations --- #

In [45]:
data_neighbours

,1,2,3,4,5,6,7,8,9,10,11
2001: A Space Odyssey (1968),2001: A Space Odyssey (1968),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,E.T. the Extra-Terrestrial (1982),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),One Flew Over the Cuckoo's Nest (1975),Reservoir Dogs (1992),Monty Python's Life of Brian (1979),"Wizard of Oz, The (1939)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Blade Runner (1982),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,2001: A Space Odyssey (1968),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),Reservoir Dogs (1992),Die Hard (1988),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),E.T. the Extra-Terrestrial (1982),Terminator 2: Judgment Day (1991),Pulp Fiction (1994)
"City of Lost Children, The (Cité des enfants perdus, La) (1995)","City of Lost Children, The (Cité des enfants p...",Blade Runner (1982),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),2001: A Space Odyssey (1968),Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),Star Wars: Episode IV - A New Hope (1977),Monty Python's Life of Brian (1979),Star Wars: Episode V - The Empire Strikes Back...,Léon: The Professional (a.k.a. The Professiona...,"Usual Suspects, The (1995)"
Clerks (1994),Clerks (1994),Pulp Fiction (1994),"Usual Suspects, The (1995)",Reservoir Dogs (1992),Seven (a.k.a. Se7en) (1995),"Silence of the Lambs, The (1991)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Monty Python and the Holy Grail (1975),"Shawshank Redemption, The (1994)",Terminator 2: Judgment Day (1991),Star Wars: Episode V - The Empire Strikes Back...
Die Hard (1988),Die Hard (1988),Star Wars: Episode V - The Empire Strikes Back...,Terminator 2: Judgment Day (1991),Star Wars: Episode IV - A New Hope (1977),E.T. the Extra-Terrestrial (1982),Blade Runner (1982),Raiders of the Lost Ark (Indiana Jones and the...,Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),"Silence of the Lambs, The (1991)",Seven (a.k.a. Se7en) (1995)
Dragonheart (1996),Dragonheart (1996),Star Wars: Episode IV - A New Hope (1977),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Rumble in the Bronx (Hont faan kui) (1995),Jumanji (1995),Terminator 2: Judgment Day (1991),"Mask, The (1994)",Star Wars: Episode V - The Empire Strikes Back...,Die Hard (1988),E.T. the Extra-Terrestrial (1982),Interview with the Vampire: The Vampire Chroni...
E.T. the Extra-Terrestrial (1982),E.T. the Extra-Terrestrial (1982),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode IV - A New Hope (1977),"Wizard of Oz, The (1939)",Die Hard (1988),Blade Runner (1982),2001: A Space Odyssey (1968),Monty Python and the Holy Grail (1975),Terminator 2: Judgment Day (1991),"Fish Called Wanda, A (1988)","Silence of the Lambs, The (1991)"
Escape to Witch Mountain (1975),Escape to Witch Mountain (1975),"Wizard of Oz, The (1939)",E.T. the Extra-Terrestrial (1982),Jumanji (1995),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode IV - A New Hope (1977),"Fish Called Wanda, A (1988)",Die Hard (1988),Dragonheart (1996),Monty Python and the Holy Grail (1975),"Mask, The (1994)"
"Fish Called Wanda, A (1988)","Fish Called Wanda, A (1988)",Monty Python's Life of Brian (1979),Monty Python and the Holy Grail (1975),E.T. the Extra-Terrestrial (1982),Die Hard (1988),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,2001: A Space Odyssey (1968),Star Wars: Episode IV - A New Hope (1977),"Wizard of Oz, The (1939)",Platoon (1986)
Interview with the Vampire: The Vampire Chronicles (1994),Interview with the Vampire: The Vampire Chroni...,Seven (a.k.a. Se7en) (1995),Pulp Fiction (1994),"Mask, The (1994)","Silence of the Lambs, The (1991)",Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Shawshank Redemption, The (1994)",Jumanji (1995),"Usual Suspects, The (1995)",Léon: The Professional (a.k.a. The Professiona...


In [46]:
recommend = data_neighbours.iloc[:,1:]

In [47]:
recommend.reset_index(inplace=True)

In [48]:
recommend.head()

,index,2,3,4,5,6,7,8,9,10,11
0,2001: A Space Odyssey (1968),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,E.T. the Extra-Terrestrial (1982),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),One Flew Over the Cuckoo's Nest (1975),Reservoir Dogs (1992),Monty Python's Life of Brian (1979),"Wizard of Oz, The (1939)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
1,Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,2001: A Space Odyssey (1968),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),Reservoir Dogs (1992),Die Hard (1988),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),E.T. the Extra-Terrestrial (1982),Terminator 2: Judgment Day (1991),Pulp Fiction (1994)
2,"City of Lost Children, The (Cité des enfants p...",Blade Runner (1982),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),2001: A Space Odyssey (1968),Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),Star Wars: Episode IV - A New Hope (1977),Monty Python's Life of Brian (1979),Star Wars: Episode V - The Empire Strikes Back...,Léon: The Professional (a.k.a. The Professiona...,"Usual Suspects, The (1995)"
3,Clerks (1994),Pulp Fiction (1994),"Usual Suspects, The (1995)",Reservoir Dogs (1992),Seven (a.k.a. Se7en) (1995),"Silence of the Lambs, The (1991)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Monty Python and the Holy Grail (1975),"Shawshank Redemption, The (1994)",Terminator 2: Judgment Day (1991),Star Wars: Episode V - The Empire Strikes Back...
4,Die Hard (1988),Star Wars: Episode V - The Empire Strikes Back...,Terminator 2: Judgment Day (1991),Star Wars: Episode IV - A New Hope (1977),E.T. the Extra-Terrestrial (1982),Blade Runner (1982),Raiders of the Lost Ark (Indiana Jones and the...,Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),"Silence of the Lambs, The (1991)",Seven (a.k.a. Se7en) (1995)


In [49]:
recommend['index'].unique()

array(['2001: A Space Odyssey (1968)', 'Blade Runner (1982)',
       'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
       'Clerks (1994)', 'Die Hard (1988)', 'Dragonheart (1996)',
       'E.T. the Extra-Terrestrial (1982)',
       'Escape to Witch Mountain (1975)', 'Fish Called Wanda, A (1988)',
       'Interview with the Vampire: The Vampire Chronicles (1994)',
       'Jumanji (1995)',
       'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
       'Mask, The (1994)', 'Monty Python and the Holy Grail (1975)',
       "Monty Python's Life of Brian (1979)",
       "One Flew Over the Cuckoo's Nest (1975)", 'Platoon (1986)',
       'Pulp Fiction (1994)',
       'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
       'Reservoir Dogs (1992)', 'Rob Roy (1995)',
       'Rumble in the Bronx (Hont faan kui) (1995)',
       'Seven (a.k.a. Se7en) (1995)', 'Shawshank Redemption, The (1994)',
       'Silence of the Lambs, The (199

In [50]:
recommend

,index,2,3,4,5,6,7,8,9,10,11
0,2001: A Space Odyssey (1968),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,E.T. the Extra-Terrestrial (1982),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),One Flew Over the Cuckoo's Nest (1975),Reservoir Dogs (1992),Monty Python's Life of Brian (1979),"Wizard of Oz, The (1939)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
1,Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,2001: A Space Odyssey (1968),Star Wars: Episode IV - A New Hope (1977),Monty Python and the Holy Grail (1975),Reservoir Dogs (1992),Die Hard (1988),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),E.T. the Extra-Terrestrial (1982),Terminator 2: Judgment Day (1991),Pulp Fiction (1994)
2,"City of Lost Children, The (Cité des enfants p...",Blade Runner (1982),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),2001: A Space Odyssey (1968),Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),Star Wars: Episode IV - A New Hope (1977),Monty Python's Life of Brian (1979),Star Wars: Episode V - The Empire Strikes Back...,Léon: The Professional (a.k.a. The Professiona...,"Usual Suspects, The (1995)"
3,Clerks (1994),Pulp Fiction (1994),"Usual Suspects, The (1995)",Reservoir Dogs (1992),Seven (a.k.a. Se7en) (1995),"Silence of the Lambs, The (1991)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Monty Python and the Holy Grail (1975),"Shawshank Redemption, The (1994)",Terminator 2: Judgment Day (1991),Star Wars: Episode V - The Empire Strikes Back...
4,Die Hard (1988),Star Wars: Episode V - The Empire Strikes Back...,Terminator 2: Judgment Day (1991),Star Wars: Episode IV - A New Hope (1977),E.T. the Extra-Terrestrial (1982),Blade Runner (1982),Raiders of the Lost Ark (Indiana Jones and the...,Reservoir Dogs (1992),Monty Python and the Holy Grail (1975),"Silence of the Lambs, The (1991)",Seven (a.k.a. Se7en) (1995)
5,Dragonheart (1996),Star Wars: Episode IV - A New Hope (1977),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Rumble in the Bronx (Hont faan kui) (1995),Jumanji (1995),Terminator 2: Judgment Day (1991),"Mask, The (1994)",Star Wars: Episode V - The Empire Strikes Back...,Die Hard (1988),E.T. the Extra-Terrestrial (1982),Interview with the Vampire: The Vampire Chroni...
6,E.T. the Extra-Terrestrial (1982),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode IV - A New Hope (1977),"Wizard of Oz, The (1939)",Die Hard (1988),Blade Runner (1982),2001: A Space Odyssey (1968),Monty Python and the Holy Grail (1975),Terminator 2: Judgment Day (1991),"Fish Called Wanda, A (1988)","Silence of the Lambs, The (1991)"
7,Escape to Witch Mountain (1975),"Wizard of Oz, The (1939)",E.T. the Extra-Terrestrial (1982),Jumanji (1995),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode IV - A New Hope (1977),"Fish Called Wanda, A (1988)",Die Hard (1988),Dragonheart (1996),Monty Python and the Holy Grail (1975),"Mask, The (1994)"
8,"Fish Called Wanda, A (1988)",Monty Python's Life of Brian (1979),Monty Python and the Holy Grail (1975),E.T. the Extra-Terrestrial (1982),Die Hard (1988),Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,2001: A Space Odyssey (1968),Star Wars: Episode IV - A New Hope (1977),"Wizard of Oz, The (1939)",Platoon (1986)
9,Interview with the Vampire: The Vampire Chroni...,Seven (a.k.a. Se7en) (1995),Pulp Fiction (1994),"Mask, The (1994)","Silence of the Lambs, The (1991)",Terminator 2: Judgment Day (1991),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Shawshank Redemption, The (1994)",Jumanji (1995),"Usual Suspects, The (1995)",Léon: The Professional (a.k.a. The Professiona...


In [51]:
### reccomend movie like a 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)'
recommend[recommend['index']=='Twelve Monkeys (a.k.a. 12 Monkeys) (1995)']

,index,2,3,4,5,6,7,8,9,10,11
28,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Pulp Fiction (1994),Seven (a.k.a. Se7en) (1995),Terminator 2: Judgment Day (1991),"Usual Suspects, The (1995)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Blade Runner (1982),"Shawshank Redemption, The (1994)",Star Wars: Episode V - The Empire Strikes Back...,Léon: The Professional (a.k.a. The Professiona...


In [54]:
recommend.loc[recommend['index'].str.contains("Silence of the Lambs", case=False)]

,index,2,3,4,5,6,7,8,9,10,11
24,"Silence of the Lambs, The (1991)",Pulp Fiction (1994),"Shawshank Redemption, The (1994)",Seven (a.k.a. Se7en) (1995),Terminator 2: Judgment Day (1991),"Usual Suspects, The (1995)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...,"Mask, The (1994)",Reservoir Dogs (1992)


In [ ]:
# prevent session loss connecting
i = 1
while i > 0 :
  # print(i)
  if i == 1000000000000000000000000000000000000000000000:
    break
  i += 1